Nota basada en [liga](https://www.dropbox.com/s/jwu8lu4r14pb7ut/3.2.1.Sistemas_de_ecuaciones_lineales_eliminacion_Gaussiana_y_factorizacion_LU.pdf?dl=0)

# Factorización LU

Esta factorización se puede obtener a partir del método de la eliminación Gaussiana. Ver [Gaussian elimination](https://en.wikipedia.org/wiki/Gaussian_elimination). 

Entre sus ventajas se encuentra el poder usarse para resolver múltiples SEL una vez calculada. Por ejemplo, si queremos resolver $t$ SEL $Ax = b_1, Ax=b_2, \dots, Ax=b_t$ con eliminación Gaussiana resultarían $\mathcal{O}(\frac{2}{3}tn^3)$ flops. Si usamos factorización $LU$ entonces factorizamos $A=LU$ con un costo de $\mathcal{O}(\frac{2}{3}n^3)$ flops y posteriormente resolvemos los $2t$ SEL triangulares con los métodos de sustitución hacia delante y hacia atrás. Lo anterior generarían $\mathcal{O}(\frac{2}{3}n^3) + \mathcal{O}(2tn^2)$ flops que es menos costoso que con eliminación Gaussiana si $t>1$.

# Factorización LU con pivoteo parcial

A la eliminación Gaussiana y a la factorización LU se les tiene que añadir una búsqueda de elementos con magnitud más grande para tener la propiedad de estabilidad numérica\* bajo el redondeo en un SPF (ver [1.2.Sistema_de_punto_flotante](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/I.computo_cientifico/1.2.Sistema_de_punto_flotante.ipynb), [1.3.Condicion_de_un_problema_y_estabilidad_de_un_algoritmo](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/I.computo_cientifico/1.3.Condicion_de_un_problema_y_estabilidad_de_un_algoritmo.ipynb)). Tal búsqueda se puede realizar por renglones, por columnas o en ambas. A la búsqueda por renglones se le llama **pivoteo parcial**\*.

\*Aunque al añadir el pivoteo parcial a la eliminación Gaussiana o a la factorización LU, se obtienen métodos que en general son númericamente estables bajo el redondeo, no lo son en la totalidad de ejemplos. Hay ejemplos construídos que destruyen su estabilidad numérica. Sin embargo, tales ejemplos no surgen en la práctica (esto se ha comprobado a través de la historia en el uso de éste método)

\*El nombre de pivoteo parcial se utiliza por la búsqueda de **pivotes** que son los elementos de la matriz por los cuales la eliminación Gaussiana realiza la fase de eliminación. El nombre "parcial" se utiliza para diferenciarla del pivoteo completo en el que se buscan los pivotes tanto en renglones como en columnas. En el pivoteo parcial la búsqueda se hace por renglones.

**Teorema**

Si $A \in  \mathbb{R}^{n \times n}$ es no singular, existen matrices $P, L, U$ de permutación, triangular inferior con $1$'s en la diagonal y triangular superior invertible respectivamente tales que $PA = LU$.

**Algoritmo gaxpy LU con pivoteo parcial**


El pivoteo parcial se implementa como una búsqueda por la entrada de máxima magnitud por renglones y un intercambio de renglones con el índice donde ocurre tal máximo. 

El algoritmo utiliza un vector $piv \in \mathbb{R}^{n-1}$ (obsérvese que tiene dimensión $n-1$) para codificar a las matrices de permutación que permiten realizar el intercambio de renglones. Una matriz de permutación es aquella que tiene exactamente un $1$ en cada renglón y columna. Por ejemplo:

$$
P=
\left[ \begin{array}{cccc}
0 &1 &0 &0\\
0 &0 &0 &1\\
0 &0 &1 &0\\
1 &0 &0 &0
\end{array}
\right]
$$

Si se desea realizar la multiplicación $Px$ con $x=(x_1,x_2,x_3,x_4)^T = (0,-2,3,1)^T$, se codifica con un vector $piv \in \mathbb{R}^{n}$ (obsérvese que tiene dimensión $n$) con entradas iguales a:

$$
piv=
\left[ \begin{array}{c}
2\\
4\\
3\\
1
\end{array}
\right]
$$

y la multiplicación de  se realiza con el código:

```
Para k de 1 a n
 intercambiar posición k de x con posición k de piv
```

En el ejemplo anterior $Px = (x_2,x_4,x_3,x_1)^T$:

$$
Px=
\begin{array}{l}
\left[ \begin{array}{cc}
0 &1 &0 &0\\
0 &0 &0 &1\\
0 &0 &1 &0\\
1 &0 &0 &0
\end{array}
\right] \cdot 
\left[\begin{array}{c}
x_1\\
x_2\\
x_3\\
x_4
\end{array}
\right] =
\left[\begin{array}{c}
x_2\\
x_4\\
x_3\\
x_1
\end{array}
\right] =
\left[\begin{array}{c}
-2\\
1\\
3\\
0
\end{array}
\right]
\end{array}
$$

La multiplicación entre matrices de permutación se puede codificar considerando $piv \in \mathbb{R}^{n-1}$ (obsérvese que tiene dimensión $n-1$) por ejemplo si 

$$P_1=
\left[\begin{array}{ccc}
0 & 0 & 1\\
0 & 1 & 0\\
1 & 0 & 0 
\end{array}
\right]
P_2=
\left[\begin{array}{ccc}
1 & 0 & 0\\
0 & 0 & 1\\
0 & 1 & 0 
\end{array}
\right]
$$

y se desea realizar la multiplicación: $P = P_2P_1$:

$$P=P_2P_1 = 
\left[\begin{array}{ccc}
1 & 0 & 0\\
0 & 0 & 1\\
0 & 1 & 0 
\end{array}
\right]
\left[\begin{array}{ccc}
0 & 0 & 1\\
0 & 1 & 0\\
1 & 0 & 0 
\end{array}
\right] =
\left[\begin{array}{ccc}
0 & 0 & 1\\
1 & 0 & 0\\
0 & 1 & 0 
\end{array}
\right]
$$



entonces $piv=(3,3)^T$ representando la primera posición $3$ un intercambio entre el primer renglón de la matriz identidad (matriz con ceros en todas sus entradas excepto en la diagonal que tiene $1$'s) con el tercer renglón de la matriz identidad. La segunda posición de $piv$ igual a $3$ representa un intercambio entre el segundo renglón de la identidad con el tercer renglón:

Primera entrada de $piv$ es $3$ entonces: 

$$
I_3=
\left[\begin{array}{ccc}
1 & 0 & 0\\
0 & 1 & 0\\
0 & 0 & 1 
\end{array}
\right] \underset{3}{\overset{1}{\longleftrightarrow}} P_1=
\left[\begin{array}{ccc}
0 & 0 & 1\\
0 & 1 & 0\\
1 & 0 & 0 
\end{array}
\right]
$$

y segunda entrada de $piv$ es $3$ entonces partiendo de la matriz anterior:

$$
P_1=
\left[\begin{array}{ccc}
0 & 0 & 1\\
0 & 1 & 0\\
1 & 0 & 0 
\end{array}
\right] \underset{3}{\overset{2}{\longleftrightarrow}} P=
\left[\begin{array}{ccc}
0 & 0 & 1\\
1 & 0 & 0\\
0 & 1 & 0\\
\end{array}
\right]
$$

que es el resultado previamente calculado $P_2P_1$.

A continuación se presenta el algoritmo para su cáclulo en una versión gaxpy, ver [3.1.Vectorizacion_BLAS_y_el_uso_del_cache_eficientemente](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/III.computo_matricial/3.1.Vectorizacion_BLAS_y_el_uso_del_cache_eficientemente.ipynb) para definición de la operación *gaxpy*.

**Algoritmo** (se considera que los índices de los *arrays* inician en el valor $1$ y no en el $0$)

Entrada: $A \in \mathbb{R}^{n \times n}$.

Salida: factores $L, U \in \mathbb{R}^{n \times n}$, vector $piv \in \mathbb{R}^{n}$. El vector $piv$ se utiliza para construir si se desea $P$.

Inicializar L con matriz identidad. Inicializar U como una matriz de ceros.

* Para $j$ de $1$ a $n$ hacer:

    * Si $j$ es 1 definir $v = A(:,1)$ # v es la primer columna de $A$. En otro caso:

        * Calcular $\hat{a} = P_{j-1} \cdots P_1 A(:,j)$ #aquí se utiliza al vector $piv$ para evitar hacer multiplicaciones de matriz vector, sólo se realizan intercambio de renglones

        * Resolver $L(1:j-1,1:j-1)z = \hat{a}(1:j-1)$ para $z \in \mathbb{R}^{j-1}$ por sustitución hacia delante
        
        * Definir $U(1:j-1,j) = z$
        
        * Definir $v(j:n) = \hat{a}(j:n) - L(j:n, 1:j-1)*z

    * Determinar $\mu$ con $ j \leq \mu \leq n$ tal que $|v(\mu)| = ||v(j:n)||_\infty$
    
    * Definir $piv(j)=\mu$ # piv es un vector de tamaño $n-1$
    
    * $v(j) \leftrightarrow v(\mu)$ # $\leftrightarrow$ significa intercambiar $j$-ésima entrada de $v$ por $\mu$-ésima entrada de $v$.
    
    * $L(j,1:j-1) \leftrightarrow L(\mu,1:j-1)$ #$\leftrightarrow$ significa intercambio
    
    * Definir $U(j,j) = v(j)$.
    
    * Si $v(j) \neq 0 $ definir $L(j+1:n,j) = v(j+1:n)/v(j)$
    